<center><h1>Clustering across fundamental region boundaries</center></h1>

We start with our imports.

In [1]:
%matplotlib qt

# Important external dependencies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# orix dependencies 
from orix.quaternion.orientation import Orientation, Misorientation
from orix.quaternion.rotation import Rotation
from orix.quaternion.symmetry import Oh
from orix.quaternion.orientation_region import OrientationRegion
from orix.vector.neo_euler import AxAngle
from orix import plot

# Colorisation & Animation
from skimage.color import label2rgb
from matplotlib.colors import to_rgb, to_hex
import matplotlib.animation as animation

plt.rc('font', size=6)

This notebook uses artificial data, which we create in the cell below.

In [2]:
# Set up data
d1 = Orientation.random_vonmises(50, alpha=50)
d2_0 = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 0, 0), np.pi/4))
d2 = Orientation.random_vonmises(50, alpha=50, reference=d2_0)
d3_0 = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 1, 0), np.pi/3))
d3 = Orientation.random_vonmises(50, alpha=50, reference=d3_0)
dat = Orientation.stack([d1, d2, d3]).flatten().set_symmetry(Oh)

fr = OrientationRegion.from_symmetry(Oh)

# Naive cluster (with no symmetry consideration)
D = (~dat).outer(dat).angle.data
dbscan_naive = DBSCAN(0.3, 10, metric='precomputed').fit(D)
print('Labels:', np.unique(dbscan_naive.labels_))

# Cluster
D = Misorientation((~dat).outer(dat)).set_symmetry(Oh,Oh).angle.data
dbscan = DBSCAN(0.3, 20, metric='precomputed').fit(D)
print('Labels:', np.unique(dbscan.labels_))

# Generate colors
colors = [to_rgb('C{}'.format(i)) for i in range(10)]  # ['C0', 'C1', ...]
c = label2rgb(dbscan.labels_, colors=colors)
c_naive = label2rgb(dbscan_naive.labels_, colors=colors)

Labels: [0 1 2 3 4 5]
Labels: [0 1 2]


We now plot our data, on the left the colours correspond to the labels assinged by the naive implementations. On the right we see the correct solution, which takes us from 6 clusters, to three. The wire grid corresponds to the fundemental zone.

In [5]:
# Figure
fig = plt.figure(figsize=(12, 7))

# This is the left hand plot
ax1 = fig.add_subplot(121, projection='axangle', aspect='equal')
ax1.scatter(dat, c=c_naive)
ax1.plot_wireframe(fr, color='gray', alpha=0.5, linewidth=0.5, rcount=30, ccount=30)
ax1.set_axis_off()

ax1.set_xlim(-0.8, 0.8)
ax1.set_ylim(-0.8, 0.8)
ax1.set_zlim(-0.8, 0.8)

# This is the right hand plot
ax2 = fig.add_subplot(122, projection='axangle', aspect='equal')
ax2.scatter(dat, c=c)
ax2.plot_wireframe(fr, color='gray', alpha=0.5, linewidth=0.5, rcount=30, ccount=30)
ax2.set_axis_off()

ax2.set_xlim(-0.8, 0.8)
ax2.set_ylim(-0.8, 0.8)
ax2.set_zlim(-0.8, 0.8)

plt.tight_layout()

# Generate animation
def animate(angle):
    ax1.view_init(15, angle)
    ax2.view_init(15, angle)
    plt.draw()

ani = animation.FuncAnimation(fig, animate, np.linspace(75, 360+74, 720), interval=25)
plt.show()